# Загрузка Pandas и очистка данных

In [ ]:
import pandas as pd
import os
import re

In [ ]:
project_dir = r'/content/drive/My Drive/Colab Notebooks/data/u3/p3/'

df = pd.read_csv(project_dir + 'main_task.csv')

In [ ]:
df.sample(7)

In [ ]:
df.shape

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки


`Cuisine Style` — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;

`Price Range` — диапазон цен в ресторане;

`Number of Reviews` — количество отзывов о ресторане;



In [ ]:
df.columns = [ str(x).replace(' ', '_') for x in df.columns]
df.columns

In [ ]:
for item in df.columns:
    print(item, ': nan-count = ', df[item].isna().sum(), sep='')

In [ ]:
df['Cuisine_Style'] = df['Cuisine_Style'].fillna(0)

In [ ]:

df['Price_Range'] = df['Price_Range'].fillna(df['Price_Range'].value_counts().index[0])

In [ ]:
df['Number_of_Reviews'] = df['Number_of_Reviews'].fillna(df['Number_of_Reviews'].mean())

In [ ]:
for item in df.columns:
    print(item, ': nan-count = ', df[item].isna().sum(), sep='')

In [ ]:
df.info()

In [ ]:
df['Cuisine_Style'].head()

In [ ]:
def split_str_to_list(incoming):
    if incoming == 0 or len(incoming) == 0:
        return ['None']
    # print(type(incoming))
    return re.findall(r"[^'][\w\s]+[^']", incoming)

# s1 = r"['Japanese', 'Sushi', 'Asian', 'Grill', 'Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options']"
# split_str_to_list(s1)


df['Cuisine_Style'] = df['Cuisine_Style'].apply(lambda x: re.findall(r"[^'][\w\s]+[^']", str(x)))

df['Restaurant_id'] = df['Restaurant_id'].apply(lambda x: int(re.search(r'\d+', str(x)).group(0)))

df['City'] = df['City'].apply(lambda x: str(x).strip())

df['ID_TA'] = df['ID_TA'].apply(lambda x: int(re.search(r'\d+', str(x)).group(0)))

price_range_dict = dict(zip(['$$ - $$$', '$', '$$$$'], [500, 5, 5000]))
df['Price_Range'] = df['Price_Range'].replace(to_replace=price_range_dict)

In [ ]:
df.drop(['Reviews', 'URL_TA'], axis = 1, inplace=True)

In [ ]:
df.sample(7)

In [ ]:
# df['Price_Range'].value_counts().index



# WARN

In [ ]:
df.shape


In [ ]:
# Усекаем датафрейм, чтобы не переполнить ОЗУ
# df = df.sample(frac=0.25)
# df.shape

In [ ]:
# df.explode('Cuisine_Style')

In [ ]:
# df.explode('Cuisine_Style')['Cuisine_Style'].value_counts()
pd.get_dummies(df['City']).shape

In [ ]:
dfr = pd.get_dummies(df.explode('Cuisine_Style')['Cuisine_Style'])


In [ ]:
dfr.shape

In [ ]:
df.columns

In [ ]:
df0 = pd.concat([df.drop(['Cuisine_Style', 'City'], axis = 1), dfr])
# 'City', 'Price_Range', 'ID_TA'



In [ ]:
df0.shape

In [ ]:
df0.sample(7)

In [ ]:
for item in df0.columns:
    print(item, ': nan-count = ', df0[item].isna().sum(), sep='')

In [ ]:
df0.info()

In [ ]:
df1 = pd.concat([df0, pd.get_dummies(df0['City'])])


In [ ]:
df1.sample(9)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))